Используемые библиотеки

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# Дифференциальные уравнения первого порядка

Задача 1.5

Одним из основных законов теории скоростей химических реакций является закон действующих масс, согласно которому скорость химической реакции при постоянной температуре пропорциональна произведению концентраций веществ, участвующих в данный момент в реакции.

Решим следующую задачу. Два жидких химических вещества $A$ и $B$ объемом $V_{a}$ и $V_{b}$ соответственно в процессе химической реакции образуют новое жидкое химическое вещество $C$. Полагая, что температура в процессе реакции не меняется, а также что из каждых двух объемов вещества $A$ и одного объема вещества $B$ образуется три объема вещества $C$, получаем зависимость количества вещества $C$ от времени в виде дифференциального уравнения $\frac{dx}{dt} = k (V_{a} - \frac{2}{3} * x)(V_{b} - \frac{1}{3} * x)$, где $x$ - объем (в литрах) вещества $C$, $k$ - положительный коэффициент пропорционально равный $0.02$. Определить время для полного прекращения реакции, а также объем вещества $A$ и $B$ в момент прекращения реакции.

In [ ]:
# Исходные данные
Va = float(input("Введите объем вещества A (Va) в литрах: "))
Vb = float(input("Введите объем вещества B (Vb) в литрах: "))
k = 0.02  # коэффициент пропорциональности

def reaction_rate(t, x):
    """
    Функция для расчета скорости реакции dx/dt.
    """
    return k * (Va - (2 / 3) * x) * (Vb - (1 / 3) * x)

# Максимальный объем вещества C
x_max = min(3 * Va / 2, 3 * Vb)

# Решение дифференциального уравнения
sol = solve_ivp(
    reaction_rate,              # функция скорости реакции
    [0, 1e5],                   # временной интервал (достаточно большой для прекращения реакции)
    [0],                        # начальное значение x (объем C в начале реакции)
    method='RK45',              # метод численного решения
    dense_output=True           # для получения непрерывного решения
)

# Определение времени прекращения реакции
for i, x in enumerate(sol.y[0]):
    if x >= x_max:
        time_end = sol.t[i]  # время прекращения реакции
        break
else:
    time_end = sol.t[-1]

# Объемы веществ A и B в момент прекращения реакции
x_final = x_max
Va_final = Va - (2 / 3) * x_final
Vb_final = Vb - (1 / 3) * x_final

# Вывод результатов
print("Результаты:")
print(f"Время полного прекращения реакции: {time_end:.2f} секунд")
print(f"Остаточный объем вещества A: {Va_final:.2f} литров")
print(f"Остаточный объем вещества B: {Vb_final:.2f} литров")

# График
t_vals = np.linspace(0, time_end, 500)
x_vals = sol.sol(t_vals)[0]

plt.figure(figsize=(10, 6))
plt.plot(t_vals, x_vals, label="Объем вещества C")
plt.axhline(y=x_max, color='r', linestyle='--', label="Максимальный объем вещества C")
plt.xlabel("Время (с)")
plt.ylabel("Объем вещества C (литры)")
plt.title("Динамика образования вещества C")
plt.legend()
plt.grid()
plt.show()

# Системы дифференциальных уравнений

Задача 2.5

Построить фазовый портрет системы дифференциальных уравнений 

$$
\{\begin{matrix}
 \frac{dx}{dt} = -y - x * (x^{2} + y^{2} - 1) \\
 \frac{dy}{dt} = x - y * (x^{2} + y^{2} - 1)
\end{matrix}
$$

In [ ]:
# Определение системы уравнений
def system(t, z):
    x, y = z
    dxdt = -y - x * (x**2 + y**2 - 1)
    dydt = x - y * (x**2 + y**2 - 1)
    return [dxdt, dydt]

# Сетка значений для построения фазового портрета
x = np.linspace(-2, 2, 20)
y = np.linspace(-2, 2, 20)
X, Y = np.meshgrid(x, y)

# Вычисление направлений векторного поля
U = -Y - X * (X**2 + Y**2 - 1)
V = X - Y * (X**2 + Y**2 - 1)

# Нормализация векторов для корректного отображения направлений
magnitude = np.sqrt(U**2 + V**2)
U /= magnitude
V /= magnitude

# Построение фазового портрета
plt.figure(figsize=(10, 8))
plt.quiver(X, Y, U, V, color="blue", alpha=0.8)
plt.title("Фазовый портрет системы", fontsize=14)
plt.xlabel("x", fontsize=12)
plt.ylabel("y", fontsize=12)
plt.axhline(0, color='black', linewidth=0.8, linestyle='--')
plt.axvline(0, color='black', linewidth=0.8, linestyle='--')
plt.grid()
plt.show()

# Равномерно распределенная дискретная случайная величина

Задача 3.5

Рассмотрим задачу о пьяном прохожем, которую называют еще задачей о случайном блуждании. Предположим, что пьяный, стоя на углу улицы, решает прогуляться. Пусть вероятности того, что, достигнув очередного перекрестка, он пойдет на север, юг, восток или запад, одинаковы. Оценить вероятность того, что, пройдя 10 кварталов, пьяный окажется не далее двух кварталов от места, где он начал прогулку. Оценку вероятности провести на основании 1000 испытаний.


In [ ]:
# Параметры задачи
steps = 10  # Количество шагов
radius = 2  # Радиус от начальной точки
trials = 1000  # Количество испытаний

# Функция для моделирования случайного блуждания
def random_walk(steps):
    """
    Симулирует случайное блуждание.
    """
    # Направления: север (0,1), юг (0,-1), восток (1,0), запад (-1,0)
    directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
    position = np.array([0, 0])
    for _ in range(steps):
        step = directions[np.random.choice(4)]
        position += step
    return position

# Моделирование и подсчет успешных случаев
success_count = 0
for _ in range(trials):
    final_position = random_walk(steps)
    distance = np.linalg.norm(final_position, ord=2)  # Евклидово расстояние
    if distance <= radius:
        success_count += 1

# Оценка вероятности
probability = success_count / trials

# Вывод результата
print(f"Вероятность того, что пьяный окажется не далее двух кварталов: {probability:.4f}")


# Равномерно распределенная непрерывная случайная величина

Задача 4.5

Оценить интеграл $I = \int^{\pi}_{0}\sin{x*dx}$ (как показывают элементарные вычисления ), используя метод статистических испытаний, при объемах выборки $n = 10, 20, 50, 100, 1000, 10000$.


In [ ]:
# Функция, подлежащая интегрированию
def f(x):
    return np.sin(x)

# Пределы интегрирования
a, b = 0, np.pi

# Объемы выборки для метода Монте-Карло
sample_sizes = [10, 20, 50, 100, 1000, 10000]

# Вычисление интеграла методом Монте-Карло
results = []
for n in sample_sizes:
    # Генерация случайных точек в пределах интегрирования
    x_samples = np.random.uniform(a, b, n)
    # Оценка интеграла
    integral_estimate = (b - a) * np.mean(f(x_samples))
    results.append((n, integral_estimate))

# Вывод результатов
print("Результаты вычисления интеграла методом Монте-Карло:")
for n, estimate in results:
    print(f"n = {n}: I ≈ {estimate:.6f}")

# Нормально распределенная случайная величина

Задача 5.5

Случайные величины $X$ и $Y$ независимы и имеют нормальное распределение с математическим ожиданием $0$ и дисперсией $1$. Оценить $P(|X-Y|<=1)$ на основании $1000$ выборочных значений пар $(x_{i}, y_{i})$, $i=1,...,1000$.


In [ ]:
# Параметры нормального распределения
mean = 0
std_dev = 1

# Количество выборочных значений
n_samples = 1000

# Генерация выборок для X и Y
X = np.random.normal(mean, std_dev, n_samples)
Y = np.random.normal(mean, std_dev, n_samples)

# Вычисление |X - Y| и подсчет случаев, удовлетворяющих условию
absolute_differences = np.abs(X - Y)
success_count = np.sum(absolute_differences <= 1)

# Оценка вероятности
probability_estimate = success_count / n_samples

# Вывод результата
print(f"Оценка вероятности P(|X - Y| <= 1): {probability_estimate:.4f}")

# Экспоненциально распределенная случайная величина

Задача 6.5

Пусть в течение суток длительность интервала времени между очередными поступлениями трамваев на остановку распределена экспоненциально с математическим ожиданием равным 15 минутам. Один раз в сутки пассажир приходит на остановку в случайный момент времени суток (равномерное распределение длительности) и ожидает очередной трамвай в течение времени $\Delta$. Провести 1000 испытаний с моделью и оценить математическое ожидание $\Delta$.


In [ ]:
# Параметры задачи
mean_interval = 15  # Средний интервал между трамваями (в минутах)
rate = 1 / mean_interval  # Параметр экспоненциального распределения
n_trials = 1000  # Количество испытаний

# Симуляция
waiting_times = []
for _ in range(n_trials):
    # Момент прихода пассажира (в минутах с начала суток)
    passenger_arrival = np.random.uniform(0, 1440)  # 1440 минут в сутках

    # Генерация интервалов между трамваями в течение суток
    tram_arrivals = [0]
    while tram_arrivals[-1] < 1440:
        tram_arrivals.append(tram_arrivals[-1] + np.random.exponential(scale=mean_interval))

    # Удаляем последний элемент, если он больше 1440 минут
    tram_arrivals = np.array(tram_arrivals[:-1])

    # Находим время ожидания
    next_tram = tram_arrivals[tram_arrivals >= passenger_arrival].min()
    waiting_time = next_tram - passenger_arrival
    waiting_times.append(waiting_time)

# Оценка математического ожидания времени ожидания
expected_waiting_time = np.mean(waiting_times)

# Вывод результата
print(f"Оценка математического ожидания времени ожидания (\u0394): {expected_waiting_time:.2f} минут")
